In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import datetime as dt
import import_ipynb
from function_log import new_compare

importing Jupyter notebook from function_log.ipynb


In [29]:
import SALib as SA
from SALib.sample import morris
from SALib.analyze import morris

In [3]:
data = pd.read_csv('../Mongoose Experiments/new_data.csv')
data.loc[:,'Date'] = pd.to_datetime(data.Date)
data.head()

,Unnamed: 0,Date,tdate,tdint,quarter,T,A,S,J,week,Troop
0,0,2008-05-11,0.526027,-1.0,3.0,31.0,NaN,NaN,NaN,28,CCH
1,1,2008-05-18,0.545205,-1.0,3.0,34.0,NaN,NaN,NaN,29,CCH
2,2,2008-05-25,0.564384,-1.0,3.0,35.0,NaN,NaN,NaN,30,CCH
3,3,2008-06-01,0.583562,-1.0,3.0,37.0,NaN,NaN,NaN,31,CCH
4,4,2008-06-08,0.602740,-1.0,3.0,37.0,NaN,NaN,NaN,32,CCH


In [4]:
wd = data.loc[(data.Troop == 'CCH')&(data.tdint==0)]
wd.head()

,Unnamed: 0,Date,tdate,tdint,quarter,T,A,S,J,week,Troop
25,25,2008-11-02,0.005479,0.0,1.0,15.0,1.0,NaN,NaN,1,CCH
26,26,2008-11-09,0.024658,0.0,1.0,14.0,4.0,NaN,NaN,2,CCH
27,27,2008-11-16,0.043836,0.0,1.0,14.0,6.0,NaN,NaN,3,CCH
28,28,2008-11-23,0.063014,0.0,1.0,14.0,9.0,NaN,NaN,4,CCH
29,29,2008-11-30,0.082192,0.0,1.0,13.0,12.0,NaN,NaN,5,CCH


In [5]:
def model_eval(fusion = 0.056,dispersal=0.05,b_j = 0.33,d_j = 0.34,d_s = 0.25,d_a = 0.034,t_p = 0.56,por = 0.05,fsr = 0.29,ae_thresh = 6,fis_thresh = 62,lit_size = 3):
    
    res_df = new_compare(wd,iters=25,
                        fusion = fusion,
                        dispersal=dispersal,
                         b_j = b_j,
                         d_j = d_j,
                         d_s = d_s,
                         d_a = d_a,
                         t_p = t_p,
                         por = por,
                         fsr = fsr,
                         ae_thresh = round(ae_thresh),
                         fis_thresh = round(fis_thresh),
                         lit_size = round(lit_size),
                         dfe='off')
    
    smape = res_df.loc[res_df.sMAPE<=100]
    smape = smape.loc[:,'sMAPE'].dropna()

    total = len(smape)
    good = len(smape.loc[smape <= 15])
    perc = 1-(good/total)
    perc*=100
    
    return perc


In [33]:
problem = {
    'num_vars': 12,
    'names': ['fusion','dispersal','b_j','d_j','d_s','d_a','t_p','por','fsr','ae_thresh','fis_thresh','lit_size'],
    'bounds': [[0.05,0.70],
              [0.05,0.70],
              [0.254,0.344],
              [0.278,0.412],
              [0.136,0.296],
              [0.034,0.252],
              [0.05,0.80],
              [0.05,0.80],
              [0.15,0.80],
              [4,6],
              [38,68],
              [3,5],]
}

In [42]:
param_values = SA.sample.morris.sample(problem,1000,num_levels=12)

In [ ]:
res = np.zeros([param_values.shape[0]])

for count,val in enumerate(param_values):
    res[count] = model_eval(val)

<string>:107: RuntimeWarning: divide by zero encountered in double_scalars
